## Problem Statement/Goal



## Imports

In [65]:
import pandas as pd
import matplotlib as plt
import seaborn as sns
import numpy as np
%matplotlib inline
from selenium import webdriver
import requests
from time import sleep
from bs4 import BeautifulSoup
import json

from sklearn.model_selection import train_test_split

# Selenium

# Translating Selenium into Requests & Soup

In [66]:
def get_html(url):
    return BeautifulSoup(requests.get(url).content, 'lxml')

In [67]:
# Function to specifically spit out a list of tuples
# tuple[0] is the url of the topic
# tuple[1] is the number of pages in the topic
def urls_with_numbers(forum_page_url):
#current_forum_page = 'https://us.battle.net/forums/en/overwatch/22813879/?page='+ str(forum_page)
    forum_soup = get_html(forum_page_url)                                       # Input URL get Soup
    topic_url_list = ['https://us.battle.net' + topic.attrs['href'] for topic in forum_soup.find_all(attrs={'class': "ForumTopic"})]
    topic_tuples = []                                                           # Above is list of urls of topics in forum page
    count = 0                                                                   # Instantiating empty list and starting count
    for forum_topic in forum_soup.find_all(attrs={'class': "ForumTopic"}):      # for i in all th3e forum topic infos
        posts_num = json.loads(forum_topic.attrs['data-forum-topic'])['lastPosition'] # Turning each individual info bit into a dict
        topic_pages = posts_num//20 + 1                     # Number of posts in the Topic divided by num allowed per page +1 for 1st
        if posts_num%20 != 0 and posts_num > 20:            # If there's a remainder page
            topic_pages += 1                                # Add remainder page
        topic_tuples.append((topic_url_list[count],topic_pages)) # Add the url to the page amount in tuples
        count += 1                                          # Keep track of the count
    return topic_tuples                                     # Returns list of tuples


In [68]:
def save_posts(list_of_dicts, saver, forum_page):
    return pd.DataFrame(list_of_dicts, columns = ['text',
                            'date',
                            'ids_dict',
                            'post_num',
                            'auth_posts',
                            'prof_link',
                            'title',
                            'forum_page',
                            'stqatuses',
                            'topic_url']).to_csv(saver + str(forum_page), index=False)

In [ ]:
words = []
dates = []
TopicPosts = []
title = []
forum_page = []
prof_link = []
auth_posts = []
topic_keys  = []
def turn_to_dict(post, words = words, 
                 dates = dates, TopicPosts = TopicPosts, 
                 title = title, forum_page = forum_page, 
                 prof_link = prof_link, auth_posts = auth_posts, topic_keys = topic_keys):
    post_dict = {"If you're seeing this": "something went wrong."}
    try:
        post_dict = {                               # Creation & Statement of dicts
            'text'      : words[post].contents,                     # Text of the post
            'date'      : dates[post].attrs['data-tooltip-content'],# Date of the (unedited) post
            'ids_dict'  : TopicPosts[post].attrs['data-topic-post'],# Author info & votes of the post
            'post_num'  : TopicPosts[post].attrs['id'] ,            # Post number in the topic
            'title'     : title,                    # Title of Topic
            'forum_page': forum_page,               # Page in the forum
            'topic_url' : topic_keys[post][0]}
    except:
        pass
    
    try:
        post_dict['statuses']   = TopicPosts[post].attrs['data-topic']
    except:
        pass
    
    try:
        post_dict['prof_link']  = prof_link[post].attrs['href'][0]
    except:
        pass
    
    try:
        post_dict['auth_posts'] = auth_posts[post][0]  # Number of posts author has made
    except:
        pass
    
    return post_dict

In [3]:
def add_attempt(dictionary, title, addition):
    try:
        dictionary[str(title)] = addition
    except:
        pass
    return

In [70]:
edt = ['\n\t\t\t\t\t\t\t\xa0(Edited)\n']

In [71]:
# Check that both forum_page 0 & 1 work without causing doubling up or other issues

In [80]:
# THIS is for turning into the Requests library
list_of_dicts = []                                              # Instantiate empty list of DICKS to FILL UP
saver = './data/full_scrapes/Overwatch_Test_'                   # The path & starting name for saving junk

for forum_page in range(101):                                 # Will go through all the forum pages specified
    topic_keys = urls_with_numbers('https://us.battle.net/forums/en/overwatch/22813879/?page='+ str(forum_page))
    if forum_page % 100 == 0:
        save_posts(list_of_dicts, saver, forum_page) 
    for topic_stats in topic_keys:                              # For every title in the list
        for page in range(topic_stats[1]):                      # for page # in range of # of pages in the topic
            #page += 1                                           # Add 1 to compensate for starting at 0
            if page > 1:                                        # Basically making sure it's not the first page
                URL = topic_stats[0] + '?page=' + str(page)     # This is basicslly for getting to all the pages
            else:
                URL = topic_stats[0]                            # This will be the first page
            sleep(5)
            
            # This chunk defines the lists of things that we want per row
            page_soup  = get_html(URL)                          # Soup of the page
            title      = page_soup.find(attrs={'class':'Topic-title'}).contents
            dates      = [e for e in page_soup.find_all('a',{"class" :'TopicPost-timestamp'}) if e.contents !=edt]
            words      = page_soup.find_all('div',{"class" :'TopicPost-bodyContent'})
            TopicPosts = page_soup.find_all(attrs = {"class" :'TopicPost'})
            auth_posts = [e.contents for e in page_soup.find_all('a',attrs = {'class' :'Author-posts'})]
            prof_link  = page_soup.find_all(attrs = {"class" :'Author-avatar '})
            print("F-Page:", forum_page, "Posts/Topic:", topic_stats[1],"Page:",page,"URL:",URL )
            if page == 1:
                print("Skipped duplicate page")
            else:
                for post in range(len(words)):                           # For each post that's present
                    list_of_dicts.append(turn_to_dict(post, words, 
                 dates, TopicPosts, title, forum_page, 
                 prof_link, auth_posts, topic_keys))        # Add the dict of post to list of posts
        print("END F-page:", forum_page,"Topic Page:", page, "Last URL:", URL) # Sanity check
df = save_posts(list_of_dicts, saver, forum_page)               # Should both save the data & create a df to see

F-Page: 0 Posts/Topic: 1 Page: 0 URL: https://us.battle.net/forums/en/overwatch/topic/20744265634
END F-page: 0 Topic Page: 0 Last URL: https://us.battle.net/forums/en/overwatch/topic/20744265634
F-Page: 0 Posts/Topic: 1 Page: 0 URL: https://us.battle.net/forums/en/overwatch/topic/20744215778
END F-page: 0 Topic Page: 0 Last URL: https://us.battle.net/forums/en/overwatch/topic/20744215778
F-Page: 0 Posts/Topic: 238 Page: 0 URL: https://us.battle.net/forums/en/overwatch/topic/20761088072
F-Page: 0 Posts/Topic: 238 Page: 1 URL: https://us.battle.net/forums/en/overwatch/topic/20761088072
Skipped duplicate page
F-Page: 0 Posts/Topic: 238 Page: 2 URL: https://us.battle.net/forums/en/overwatch/topic/20761088072?page=2
F-Page: 0 Posts/Topic: 238 Page: 3 URL: https://us.battle.net/forums/en/overwatch/topic/20761088072?page=3
F-Page: 0 Posts/Topic: 238 Page: 4 URL: https://us.battle.net/forums/en/overwatch/topic/20761088072?page=4
F-Page: 0 Posts/Topic: 238 Page: 5 URL: https://us.battle.net/fo

KeyboardInterrupt: 

In [82]:
df = save_posts(list_of_dicts, saver, forum_page)

In [87]:
pd.DataFrame(list_of_dicts, columns = ['text',
                            'date',
                            'ids_dict',
                            'post_num',
                            'auth_posts',
                            'prof_link',
                            'title',
                            'forum_page',
                            'statuses',
                            'topic_url'])

,text,date,ids_dict,post_num,auth_posts,prof_link,title,forum_page,statuses,topic_url
0,"[Welcome to the , <strong>General Discussion</...",05/23/2016 09:32 PM,"{""id"":""207423780179"",""valueVoted"":0,""rank"":{""v...",post-1,NaN,NaN,[Welcome to General Discussion - Please Read!],0,"{ ""sticky"":""true"",""featured"":""false"",""locked"":...",https://us.battle.net/forums/en/overwatch/topi...
1,"[In addition to the forum , <a class=""bml-link...",05/23/2016 09:33 PM,"{""id"":""207423980450"",""valueVoted"":0,""rank"":{""v...",post-2,NaN,NaN,[Welcome to General Discussion - Please Read!],0,NaN,https://us.battle.net/forums/en/overwatch/topi...
2,[To help us get the most of out of your feedba...,05/23/2016 09:36 PM,"{""id"":""207423481262"",""valueVoted"":0,""rank"":{""v...",post-1,NaN,NaN,[Bug Report and Technical Support Forums],0,"{ ""sticky"":""true"",""featured"":""false"",""locked"":...",https://us.battle.net/forums/en/overwatch/topi...
3,"[Hi everyone, , <br/>, <br/>, We've continued ...",01/30/2018 09:21 PM,"{""id"":""207594900856"",""valueVoted"":0,""rank"":{""v...",post-1,[ 368 posts ],https://playoverwatch.com/en-us/career/pc/us/M...,"[[Feedback Thread] Mercy Updates - Jan 30, 2018]",0,"{ ""sticky"":""true"",""featured"":""false"",""locked"":...",https://us.battle.net/forums/en/overwatch/topi...
4,[Please try to see the Valkyrie rework didn't ...,01/30/2018 09:25 PM,"{""id"":""207595100830"",""valueVoted"":0,""rank"":{""v...",post-2,[ 17 posts ],https://playoverwatch.com/en-us/career/pc/us/O...,"[[Feedback Thread] Mercy Updates - Jan 30, 2018]",0,NaN,https://us.battle.net/forums/en/overwatch/topi...
5,[Ban-hammer much? Surprised you guys didnt re...,01/30/2018 09:26 PM,"{""id"":""207592902925"",""valueVoted"":0,""rank"":{""v...",post-3,[ 7 posts ],https://playoverwatch.com/en-us/career/pc/us/M...,"[[Feedback Thread] Mercy Updates - Jan 30, 2018]",0,NaN,https://us.battle.net/forums/en/overwatch/topi...
6,[You guys are joking with this right? The othe...,01/30/2018 09:27 PM,"{""id"":""207593602686"",""valueVoted"":0,""rank"":{""v...",post-4,[ 17 posts ],https://playoverwatch.com/en-us/career/pc/us/A...,"[[Feedback Thread] Mercy Updates - Jan 30, 2018]",0,NaN,https://us.battle.net/forums/en/overwatch/topi...
7,[tldr: Mercy has no Ult anymore],01/30/2018 09:28 PM,"{""id"":""207593402413"",""valueVoted"":0,""rank"":{""v...",post-5,[ 353 posts ],https://playoverwatch.com/en-us/career/pc/us/E...,"[[Feedback Thread] Mercy Updates - Jan 30, 2018]",0,NaN,https://us.battle.net/forums/en/overwatch/topi...
8,[You took in tons of feedback and still ignore...,01/30/2018 09:29 PM,"{""id"":""207594302097"",""valueVoted"":0,""rank"":{""v...",post-6,[ 504 posts ],https://playoverwatch.com/en-us/career/pc/us/W...,"[[Feedback Thread] Mercy Updates - Jan 30, 2018]",0,NaN,https://us.battle.net/forums/en/overwatch/topi...
9,"[<blockquote class=""quote-public"" data-quote=""...",01/30/2018 09:29 PM,"{""id"":""207593703439"",""valueVoted"":0,""rank"":{""v...",post-7,[ 6 posts ],https://playoverwatch.com/en-us/career/pc/us/E...,"[[Feedback Thread] Mercy Updates - Jan 30, 2018]",0,NaN,https://us.battle.net/forums/en/overwatch/topi...


In [511]:
list_of_topic_links = ['https://us.battle.net' + topic.attrs['href'] for topic in resoup.find_all(attrs={'class': "ForumTopic"})]

In [23]:
test = resoup.find_all(attrs={'class': "ForumTopic"})
for i in range(len(test)):
    print('-------')
    print(i)
    posts = json.loads(test[i].attrs['data-forum-topic'])['lastPosition']
    print('posts:', posts)
    pages = posts//20 + 1
    print("pages:", pages)
    if posts%20 > 0 and posts > 21:
        pages += 1
        print("has remainder", posts%20, 'pages:', pages)
    

-------
0
posts: 1
pages: 1
-------
1
posts: 0
pages: 1
-------
2
posts: 4736
pages: 237
has remainder 16 pages: 238
-------
3
posts: 89
pages: 5
has remainder 9 pages: 6
-------
4
posts: 161
pages: 9
has remainder 1 pages: 10
-------
5
posts: 17
pages: 1
-------
6
posts: 7
pages: 1
-------
7
posts: 18
pages: 1
-------
8
posts: 14
pages: 1
-------
9
posts: 30
pages: 2
has remainder 10 pages: 3
-------
10
posts: 25
pages: 2
has remainder 5 pages: 3
-------
11
posts: 5
pages: 1
-------
12
posts: 17
pages: 1
-------
13
posts: 3
pages: 1
-------
14
posts: 9
pages: 1
-------
15
posts: 8
pages: 1
-------
16
posts: 22
pages: 2
has remainder 2 pages: 3
-------
17
posts: 12
pages: 1
-------
18
posts: 6
pages: 1
-------
19
posts: 0
pages: 1
-------
20
posts: 6244
pages: 313
has remainder 4 pages: 314
-------
21
posts: 37
pages: 2
has remainder 17 pages: 3
-------
22
posts: 14
pages: 1
-------
23
posts: 6
pages: 1
-------
24
posts: 8
pages: 1
-------
25
posts: 6
pages: 1
-------
26
posts: 0
pages

In [23]:
test = resoup.find_all(attrs={'class': "ForumTopic"})
for i in range(len(test)):
    posts = json.loads(test[i].attrs['data-forum-topic'])['lastPosition']
    pages = posts//20 + 1
    if posts%20 > 0 and posts > 21:
        pages += 1
for  in test

-------
0
posts: 1
pages: 1
-------
1
posts: 0
pages: 1
-------
2
posts: 4736
pages: 237
has remainder 16 pages: 238
-------
3
posts: 89
pages: 5
has remainder 9 pages: 6
-------
4
posts: 161
pages: 9
has remainder 1 pages: 10
-------
5
posts: 17
pages: 1
-------
6
posts: 7
pages: 1
-------
7
posts: 18
pages: 1
-------
8
posts: 14
pages: 1
-------
9
posts: 30
pages: 2
has remainder 10 pages: 3
-------
10
posts: 25
pages: 2
has remainder 5 pages: 3
-------
11
posts: 5
pages: 1
-------
12
posts: 17
pages: 1
-------
13
posts: 3
pages: 1
-------
14
posts: 9
pages: 1
-------
15
posts: 8
pages: 1
-------
16
posts: 22
pages: 2
has remainder 2 pages: 3
-------
17
posts: 12
pages: 1
-------
18
posts: 6
pages: 1
-------
19
posts: 0
pages: 1
-------
20
posts: 6244
pages: 313
has remainder 4 pages: 314
-------
21
posts: 37
pages: 2
has remainder 17 pages: 3
-------
22
posts: 14
pages: 1
-------
23
posts: 6
pages: 1
-------
24
posts: 8
pages: 1
-------
25
posts: 6
pages: 1
-------
26
posts: 0
pages

In [ ]:
# obtain topic number in touple pair with the url
# take the posts number if it's >0 add all posts to list
# then go to next page

In [267]:
#current_list = browser.get('https://us.battle.net/forums/en/overwatch/22813879/?page=5')

In [184]:
soup = BeautifulSoup(browser.page_source, 'lxml')         # Soup of words
words = soup.find_all('div',{"class" :'TopicPost-bodyContent'})
dates = soup.find_all('a',{"class" :'TopicPost-timestamp'})

print(len(words))
print(len(dates))

20
22


In [409]:
# Testing Code # 1
path_to_chromedriver = '../Garage/chromedriver'                          # Path to Chromedriver
browser = webdriver.Chrome(executable_path = path_to_chromedriver)       # Open browser
current_list = 'https://us.battle.net/forums/en/overwatch/22813879/?page=9'
browser.get(current_list)
title_list = browser.find_elements_by_class_name("ForumTopic-title")
title_list[4].click()
browser.find_element_by_class_name('Topic-title').text

TimeoutException: Message: timeout
  (Session info: chrome=71.0.3578.98)
  (Driver info: chromedriver=2.43.600229 (3fae4d0cda5334b4f533bede5a4787f7b832d052),platform=Mac OS X 10.14.0 x86_64)


In [409]:
# Testing Code # 1
path_to_chromedriver = '../Garage/chromedriver'                          # Path to Chromedriver
browser = webdriver.Chrome(executable_path = path_to_chromedriver)       # Open browser
current_list = 'https://us.battle.net/forums/en/overwatch/22813879/?page=9'
browser.get(current_list)
title_list = browser.find_elements_by_class_name("ForumTopic-title")
title_list[4].click()
browser.find_element_by_class_name('Topic-title').text

TimeoutException: Message: timeout
  (Session info: chrome=71.0.3578.98)
  (Driver info: chromedriver=2.43.600229 (3fae4d0cda5334b4f533bede5a4787f7b832d052),platform=Mac OS X 10.14.0 x86_64)


In [426]:
# Testing Code # 2
path_to_chromedriver = '../Garage/chromedriver'                          # Path to Chromedriver
browser = webdriver.Chrome(executable_path = path_to_chromedriver)       # Open browser
browser.get(current_topic + '?page=' + str(next_page))
soup = BeautifulSoup(browser.page_source, 'lxml')
#soup.find_all('div',{"class" :'TopicPost-bodyContent'})

In [140]:
# Moves down the forum, clicking on each individual title link
title_list = browser.find_elements_by_class_name("ForumTopic-title")
for i in range(len(title_list)):
    print(title_list[i])
    title_list[i].click()
    sleep(1)
    browser.back()
    title_list = browser.find_elements_by_class_name("ForumTopic-title")
    print(title_list[i])

<selenium.webdriver.remote.webelement.WebElement (session="cae02938f147fa6edb631d90ee440b6d", element="0.9641869846177666-1")>
<selenium.webdriver.remote.webelement.WebElement (session="cae02938f147fa6edb631d90ee440b6d", element="0.3229329085227737-1")>
<selenium.webdriver.remote.webelement.WebElement (session="cae02938f147fa6edb631d90ee440b6d", element="0.3229329085227737-2")>
<selenium.webdriver.remote.webelement.WebElement (session="cae02938f147fa6edb631d90ee440b6d", element="0.3916746129055908-2")>
<selenium.webdriver.remote.webelement.WebElement (session="cae02938f147fa6edb631d90ee440b6d", element="0.3916746129055908-3")>
<selenium.webdriver.remote.webelement.WebElement (session="cae02938f147fa6edb631d90ee440b6d", element="0.9025396260201077-3")>
<selenium.webdriver.remote.webelement.WebElement (session="cae02938f147fa6edb631d90ee440b6d", element="0.9025396260201077-4")>
<selenium.webdriver.remote.webelement.WebElement (session="cae02938f147fa6edb631d90ee440b6d", element="0.787543

KeyboardInterrupt: 

In [409]:
# Testing Code # 1
path_to_chromedriver = '../Garage/chromedriver'                          # Path to Chromedriver
browser = webdriver.Chrome(executable_path = path_to_chromedriver)       # Open browser
current_list = 'https://us.battle.net/forums/en/overwatch/22813879/?page=9'
browser.get(current_list)
title_list = browser.find_elements_by_class_name("ForumTopic-title")
title_list[4].click()
browser.find_element_by_class_name('Topic-title').text

TimeoutException: Message: timeout
  (Session info: chrome=71.0.3578.98)
  (Driver info: chromedriver=2.43.600229 (3fae4d0cda5334b4f533bede5a4787f7b832d052),platform=Mac OS X 10.14.0 x86_64)


In [170]:
browser.find_elements_by_class_name("TopicPost-timestamp")

[<selenium.webdriver.remote.webelement.WebElement (session="022625332a26ac2d6e8625cba583c36b", element="0.601834304827114-1")>,
 <selenium.webdriver.remote.webelement.WebElement (session="022625332a26ac2d6e8625cba583c36b", element="0.601834304827114-2")>,
 <selenium.webdriver.remote.webelement.WebElement (session="022625332a26ac2d6e8625cba583c36b", element="0.601834304827114-3")>,
 <selenium.webdriver.remote.webelement.WebElement (session="022625332a26ac2d6e8625cba583c36b", element="0.601834304827114-4")>,
 <selenium.webdriver.remote.webelement.WebElement (session="022625332a26ac2d6e8625cba583c36b", element="0.601834304827114-5")>,
 <selenium.webdriver.remote.webelement.WebElement (session="022625332a26ac2d6e8625cba583c36b", element="0.601834304827114-6")>,
 <selenium.webdriver.remote.webelement.WebElement (session="022625332a26ac2d6e8625cba583c36b", element="0.601834304827114-7")>,
 <selenium.webdriver.remote.webelement.WebElement (session="022625332a26ac2d6e8625cba583c36b", element="

In [21]:
#dfp01 = pd.read_csv('./data/Finalish Scrapes/Overwatch_24to77_posts_75.csv')
dfp02 = pd.read_csv('./data/Finalish Scrapes/Overwatch_24to77_posts_77.csv')
#dfp03 = pd.read_csv('./data/Finalish Scrapes/Overwatch_posts_100')
#dfp04 = pd.read_csv('./data/Finalish Scrapes/Overwatch_posts_125')
dfp05 = pd.read_csv('./data/Finalish Scrapes/Overwatch_posts_149')
#dfp06 = pd.read_csv('./data/Finalish Scrapes/Overwatch_SAMPLE150to1000_950')
dfp07 = pd.read_csv('./data/Finalish Scrapes/Overwatch_SAMPLE150to1000_990')
dfp08 = pd.read_csv('./data/Finalish Scrapes/Overwatch_SAMPLEposts_1500')
#dfp09 = pd.read_csv('./data/Finalish Scrapes/Overwatch_SAMPLEposts_1520')
#dfp10 = pd.read_csv('./data/Finalish Scrapes/Overwatch_SAMPLEposts_1530')
dfp11 = pd.read_csv('./data/Finalish Scrapes/Overwatch_Sample1500to5k_5000')
#dfp12 = pd.read_csv('./data/Finalish Scrapes/Overwatch_Sample5kto10k_7200')
dfp13 = pd.read_csv('./data/Finalish Scrapes/Overwatch_Sample5kto10k_7250')

In [53]:
#df_24to77     = pd.read_csv('./data/Overwatch_24to77_posts_77.csv')
#df_149        = pd.read_csv('./data/Overwatch_posts_149')          # An error skipped pages 24 to 77
df_s0to150    = pd.read_csv('./data/Overwatch_Sample_to1500_160')
df_s150to1000 = pd.read_csv('./data/Overwatch_SAMPLE150to1000_990')
df_s1500      = pd.read_csv('./data/Overwatch_SAMPLEposts_1500')
df_s1500to5k  = pd.read_csv('./data/Overwatch_Sample1500to5k_5000')
df_s5kto10k   = pd.read_csv('./data/Overwatch_Sample5kto10k_7250')

In [56]:
df_s0to150['date']      = df_s0to150['date'].map(lambda x: x.replace('\t','').replace('\n',''))
df_s150to1000['date']   = df_s150to1000['date'].map(lambda x: x.replace('\t','').replace('\n',''))
df_s1500['date']        = df_s1500['date'].map(lambda x: x.replace('\t','').replace('\n',''))
df_s1500to5k['date']    = df_s1500to5k['date'].map(lambda x: x.replace('\t','').replace('\n',''))
df_s5kto10k['date']     = df_s5kto10k['date'].map(lambda x: x.replace('\t','').replace('\n',''))

In [57]:
dfs_listed = [    
    df_s0to150,
    df_s150to1000,
    df_s1500,
    df_s1500to5k,
    df_s5kto10k]

In [86]:

for text in df_s150to1000['text']:
    try:
        if '<Removed by forum moderator for toxicity>' in text:
            print(text)
    except:
        print(text)
        #print(df_s0to150['date'])

<Removed by forum moderator for toxicity>
<Removed by forum moderator for toxicity>
<Removed by forum moderator for toxicity>
<Removed by forum moderator for toxicity>
nan
<Removed by forum moderator for toxicity>
<Removed by forum moderator for toxicity>
<Removed by forum moderator for toxicity>
<Removed by forum moderator for toxicity>


In [92]:
for df in dfs_listed:
    for text in df['text']:
        try:
            if '<Removed' in text:
                df.head(1)
                print(text)
        except:
            pass

<Removed by forum moderator for toxicity>
<Removed by forum moderator for toxicity>
<Removed by forum moderator for toxicity>
<Removed by forum moderator for toxicity>
<Removed>------------Removed by moderator for toxicity. Please adhere to the code of conduct when posting. Toxicity in all forms will result in suspension of posting privileges.
01/19/2018 08:57 AMPosted by Makenai<span class="truncated">...</span>Stop crying omfg, we're not considered as "fake acoount" we just didn't had any interest to login to the forums. we just base our opinions about this lame  no skill required hero.Does your mom know you use that kind of language online?Well she does you know<Removed>----------------------Edited by moderator for language. Please adhere to the code of conduct when posting.
<Removed by forum moderator for toxicity>
<Removed by forum moderator for toxicity>
<Removed by forum moderator for toxicity>
<Removed by forum moderator for toxicity>
<Removed by forum moderator for toxicity>
<

In [58]:
for df in dfs_listed:
    print(df['date'].value_counts())

 (Edited)       821
Feb 20          719
Feb 15          705
Feb 12          583
Feb 16          528
Feb 18          485
Feb 13          479
Feb 11          466
Feb 14          466
Feb 19          384
Feb 17          330
Feb 8           109
Feb 10           79
Feb 9            78
Jan 17           33
Dec 16           29
Nov 4, 2017      27
Jun 24, 2016     27
Jan 10           24
Nov 21, 2017     20
Feb 1            20
May 24, 2017     20
Feb 3            18
Dec 22           18
Oct 16, 2016     18
Dec 10           18
Jan 16           18
Mar 25, 2017     18
Sep 8, 2016      18
Feb 4            17
               ... 
Nov 28, 2017      1
May 26, 2017      1
Feb 2             1
May 17, 2017      1
Jul 11, 2017      1
Aug 30, 2017      1
May 26, 2016      1
Oct 14, 2016      1
Apr 27, 2017      1
Dec 1, 2017       1
Mar 3, 2017       1
Aug 24, 2016      1
Jul 19, 2017      1
Aug 30, 2016      1
Dec 2, 2016       1
Oct 10, 2016      1
Apr 19, 2017      1
Oct 22, 2017      1
Oct 13, 2016      1


In [294]:
path_to_chromedriver = '../Garage/chromedriver'                          # Path to Chromedriver
browser = webdriver.Chrome(executable_path = path_to_chromedriver)       # Open browser
list_of_dicts = []
saver = './data/full_scrapes/Overwatch_'                                 # Path to save

for forum_page in range(0,1001):                                         # For each page in chunk of posts
    current_list = 'https://us.battle.net/forums/en/overwatch/22813879/?page='+ str(forum_page)
    browser.get(current_list)                                        # Go to forum page
    title_list = browser.find_elements_by_class_name("ForumTopic-title")# Define list of topic pages to click
    if forum_page % 100 == 0:                                        # Failsafe saver per 100 pages
        save_posts(list_of_dicts, saver, forum_page)

    for i in range(len(title_list)):                                 # For every title in the list
        title_list[i].click()                                    # Click on Title i
        more_pages = True                                        # True means there's more pages
        print('start dict')
        while more_pages == True:                                # While true that there's more pages...
            title      = browser.find_element_by_class_name('Topic-title').text
            dates      = [e.get_attribute('data-tooltip-content') for e in browser.find_elements_by_class_name('TopicPost-timestamp') if e.text != ' (Edited)']
            words      = [e.text for e in browser.find_elements_by_class_name('TopicPost-bodyContent')]
            nums_dict  = [e.get_attribute('data-topic-post') for e in browser.find_elements_by_class_name('TopicPost')]
            post_num   = [e.get_attribute('id') for e in browser.find_elements_by_class_name('TopicPost')]
            auth_posts = [e.text for e in browser.find_elements_by_class_name('Author-posts') if e.get_attribute('data-toggle') == 'tooltip']
            prof_link  = [e.get_attribute('href') for e in browser.find_elements_by_class_name('Author-avatar ')]
            all_imgs   = [e.get_attribute('src') for e in browser.find_elements_by_css_selector('img')]
            auth_img   = [e for e in all_imgs if 'blznav' not in e and len(e) != 0][1:-1]
            print('dict components defined')
            for post in range(len(words)):                       # For each post in topic
                try:
                    post_dict = {                                    # Creation & Statement of dicts
                        'text'      : words[post],                   # Text of the post
                        'date'      : dates[post],                   # Date of the (unedited) post
                        'nums_dict' : nums_dict[post],               # Author info & votes of the post
                        'post_num'  : post_num[post],                # Post umber in the topic
                        'prof_link' : prof_link[post],               # Link to author profile
                        'auth_img'  : auth_img[post],                # Profile image of author
                        'title'     : title,                         # Title of Topic
                        'forum_page': forum_page                     # Page in the forum
                    }
                except:
                    pass
                try:
                    post_dict['auth_posts'] = auth_posts[post]   # Number of posts author has made
                except:
                    pass
                list_of_dicts.append(post_dict)
                print('dict created and appended')
            more_pages = click_next()                            # Clicks "NEXT" otherwise return False
            print('next clicked')
        browser.get(current_list)                                # Returns to current forum page
        sleep(0.5)                                               # Tiny safety sleep
        print('slept')
        title_list = browser.find_elements_by_class_name("ForumTopic-title") # Redefine list for freshness
save_posts(list_of_dicts, saver, forum_page)                             # Finally save the posts
df = pd.DataFrame(list_of_dicts, columns = ['text','date'])              # Return the dataframe for looking! :D

start dict
dict components defined
dict created and appended
dict created and appended
next clicked
slept
start dict
dict components defined
dict created and appended
next clicked
slept
start dict
dict components defined
dict created and appended
dict created and appended
dict created and appended
dict created and appended
dict created and appended
dict created and appended
dict created and appended
dict created and appended
dict created and appended
dict created and appended
dict created and appended
dict created and appended
dict created and appended
dict created and appended
dict created and appended
dict created and appended
dict created and appended
dict created and appended
dict created and appended
next clicked
dict components defined
dict created and appended
dict created and appended
dict created and appended
dict created and appended
dict created and appended
dict created and appended
dict created and appended
dict created and appended
dict created and appended
dict created a

MaxRetryError: HTTPConnectionPool(host='127.0.0.1', port=51282): Max retries exceeded with url: /session/e7d52587f19ee4dcdc2fe5c5868631d3/url (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x1a2ccdee80>: Failed to establish a new connection: [Errno 61] Connection refused'))

In [ ]:
def unknown_funct(title_list, list_of_dicts, browser):
    for i in range(len(title_list)):
        try:
            title_list[i].click()                                           # Click on Title i
            soup = BeautifulSoup(browser.page_source, 'lxml')               # Soup of all
            words = soup.find_all('div',{"class" :'TopicPost-bodyContent'}) # Words
            dates = soup.find_all('a',{"class" :'TopicPost-timestamp'})     # Dates
            sleep(0.5)
            
            for post in range(len(words)):
                post_dict = {                                         # Creation & Statement of dicts
                    'text' : words[post].text,
                    'date' : dates[post].text
                }
                list_of_dicts.append(post_dict)

            browser.back()                                            # Back
            title_list = browser.find_elements_by_class_name("ForumTopic-title")
        except:
            print("Woops", forum_page, i)
            save_posts(list_of_dicts, saver, forum_page)

In [186]:
dates = browser.find_elements_by_class_name('TopicPost-timestamp')
print(len(dates))
edits = []
edit_nums = []
#for element_num in range(len(dates)):
#    if "(Edited)" in dates[element_num].text:
#        print('element', dates[element_num].text)
#        edit_nums.append(element_num)

[element for element in dates if element.text not '(Edited)']
for element in dates:
    if "(Edited)" in element.:
        print('element', element.rect. text)
        #edit_nums.append(element_num)
        dates -= element
        #y = [s for s in x if len(s) == 2]
print(len(edits))
print(edits)
count = 0
#for num in edit_nums:
    
    #edits.append(dates[num].get_attribute('data-tooltip-content'))
#    dates.pop(num + count)
#    count += 1
print(edits)
print(len(dates))


for element in dates:
    print('------')
    print(element.text)
    print(element.get_attribute('data-tooltip-content'))

SyntaxError: invalid syntax (<ipython-input-186-11280649f39f>, line 10)

In [257]:
dates      = [e.get_attribute('data-tooltip-content') for e in browser.find_elements_by_class_name('TopicPost-timestamp') if e.text != ' (Edited)']
words      = [e.text for e in browser.find_elements_by_class_name('TopicPost-bodyContent')]
nums_dict  = [e.get_attribute('data-topic-post') for e in browser.find_elements_by_class_name('TopicPost')]
post_num   = [e.get_attribute('id') for e in browser.find_elements_by_class_name('TopicPost')]
auth_posts = [e.text for e in browser.find_elements_by_class_name('Author-posts') if e.get_attribute('data-toggle') == 'tooltip']
prof_link  = [e.get_attribute('href') for e in browser.find_elements_by_class_name('Author-avatar ')]
all_imgs   = [e.get_attribute('src') for e in browser.find_elements_by_css_selector('img')]
auth_img   = [e for e in all_imgs if 'blznav' not in e and len(e) != 0][1:-1]

In [136]:
elements = browser.find_elements_by_class_name('TopicPost ')
for element in elements:
    print('------')
    #print(element)
    #print(element.get_property('title'))
    #print(element.get_attribute('property'))
    print(element.get_attribute('data-topic-post'))
    print(element.get_attribute('id'))

------
{"id":"207425174325","valueVoted":0,"rank":{"voteUp":0,"voteDown":36},"author":{"id":"196469910876","name":"Dakrath"}}
post-1
------
{"id":"207425174328","valueVoted":0,"rank":{"voteUp":18,"voteDown":1},"author":{"id":"186138967094","name":"MartinRiggs"}}
post-2
------
{"id":"207425274314","valueVoted":0,"rank":{"voteUp":1,"voteDown":1},"author":{"id":"187538495100","name":"Leinhart"}}
post-3
------
{"id":"207425474302","valueVoted":0,"rank":{"voteUp":6,"voteDown":3},"author":{"id":"192868519547","name":"iHazelwood"}}
post-4
------
{"id":"207424374360","valueVoted":0,"rank":{"voteUp":15,"voteDown":0},"author":{"id":"207416589379","name":"sandwhip"}}
post-5
------
{"id":"207425374370","valueVoted":0,"rank":{"voteUp":11,"voteDown":0},"author":{"id":"186013081591","name":"RubberDucky"}}
post-6
------
{"id":"207425374378","valueVoted":0,"rank":{"voteUp":1,"voteDown":18},"author":{"id":"196469910876","name":"Dakrath"}}
post-7
------
{"id":"207425074318","valueVoted":0,"rank":{"voteUp

In [67]:
count = 0
for i in range(len(divs)):
    try:
        print(divs[i].attrs['id'])
        print(divs[i].attrs)
        divs[i].text
    except:
        count += 1

post-1
{'class': ['TopicPost', ''], 'id': 'post-1', 'data-topic-post': '{"id":"207604184492","valueVoted":0,"rank":{"voteUp":8,"voteDown":8},"author":{"id":"207417590476","name":"Pigeon"}}', 'data-topic': '{ "sticky":"false","featured":"false","locked":"false","frozen":"false","hidden":"false","pollId":"0"}'}

{'class': ['Author'], 'id': '', 'data-topic-post-body-content': 'true'}
post-2
{'class': ['TopicPost', ''], 'id': 'post-2', 'data-topic-post': '{"id":"207603084678","valueVoted":0,"rank":{"voteUp":29,"voteDown":2},"author":{"id":"182986383640","name":"timotmcc"}}'}

{'class': ['Author'], 'id': '', 'data-topic-post-body-content': 'true'}
post-3
{'class': ['TopicPost', ''], 'id': 'post-3', 'data-topic-post': '{"id":"207604384709","valueVoted":0,"rank":{"voteUp":8,"voteDown":0},"author":{"id":"207602084749","name":"Stark"}}'}

{'class': ['Author'], 'id': '', 'data-topic-post-body-content': 'true'}
post-4
{'class': ['TopicPost', ''], 'id': 'post-4', 'data-topic-post': '{"id":"2076045

In [ ]:
links = driver.find_elements_by_xpath("xpath")
links = [link.get_attribute('href') for link in driver.find_elements_by_xpath("xpath")]
for link in links:
    driver.get(link)
for x in range(len(links)):
    links[x].click()
    try:
        driver.implicitly_wait(3)
        DO something
        driver.back()
        print("Mission completed!!")
    except (ElementNotVisibleException, NoSuchElementException):
        driver.back()
        print("No action")


## Recycling Bin for Code

In [309]:
# ORIGINAL SCRAPING SAMPLING CODE
#ops = webdriver.chrome.options.Options()
#ops.add_argument('--dns-prefetch-disable')
path = '../Garage/chromedriver'                                         # Path to Chromedriver
browser = webdriver.Chrome(executable_path = path)#, options=ops)         # Open browser
list_of_dicts = []
saver = './data/full_scrapes/Overwatch_Test_' 

for forum_page in range(1,1000):
    try:
        current_list = 'https://us.battle.net/forums/en/overwatch/22813879/?page='+ str(forum_page)
        browser.get(current_list)
        title_list = browser.find_elements_by_class_name("ForumTopic-title")
        if forum_page % 100 == 0:
            save_posts(list_of_dicts, saver, forum_page)

        for i in range(len(title_list)):
            try:
                title_list[i].click()                                   # Click on Title i
                more_pages = True                                       # True means there's more pages
                #print('start dict')
                while more_pages == True:                                     # Click on Title i
                    soup = BeautifulSoup(browser.page_source, 'lxml')         # Soup of all
                    words = soup.find_all('div',{"class" :'TopicPost-bodyContent'}) # Words
                    dates = soup.find_all('a',{"class" :'TopicPost-timestamp'})     # Dates
                    for post in range(len(words)):
                        post_dict = {                                         # Creation & Statement of dicts
                            'text' : words[post].text,
                            'date' : dates[post].text
                        }
                        list_of_dicts.append(post_dict)
         
                    more_pages = click_next()                           # Clicks "NEXT" otherwise return False
                    browser.get(current_list)                                            # Back
                    title_list = browser.find_elements_by_class_name("ForumTopic-title")
            except:
                print("Woops", forum_page, i)
                save_posts(list_of_dicts, saver, forum_page)
                sleep(10)
                browser.refresh()
                browser.get(current_list)
    except:
        print("Woops", forum_page)
        save_posts(list_of_dicts, saver, forum_page)
        sleep(30)
        browser.refresh()
        browser.get(current_list)
save_posts(list_of_dicts, saver, forum_page)
df = pd.DataFrame(list_of_dicts, columns = ['text','date'])        

Woops 2 5
Woops 2


KeyboardInterrupt: 

In [314]:
# OLD CODE WITH PROBABLY TOO MANY BROWSER CALLS
ops = webdriver.chrome.options.Options()
ops.add_argument('--dns-prefetch-disable')
path = '../Garage/chromedriver'                                         # Path to Chromedriver
browser = webdriver.Chrome(executable_path = path, options=ops)         # Open browser
list_of_dicts = []
saver = './data/full_scrapes/Overwatch_Pearl_'                          # Path to save

for forum_page in range(0,1001):     # BE CAREFUL WITH THIS             # For each page in chunk of posts
    try:                                                                # Failsafe within forum page
        current_list = 'https://us.battle.net/forums/en/overwatch/22813879/?page='+ str(forum_page)
        browser.get(current_list)                                       # Go to forum page
        title_list = browser.find_elements_by_class_name("ForumTopic-title")# Define list of topic pages to click
        if forum_page % 100 == 0 & forum_page != 0:                                       # Failsafe saver per 100 pages
            save_posts(list_of_dicts, saver, forum_page)

        for i in range(len(title_list)):                                # For every title in the list
            try:                                                        # Failsafe in topic page
                title_list[i].click()                                   # Click on Title i
                more_pages = True                                       # True means there's more pages
                #print('start dict')
                #sleep(2)
                while more_pages == True:                               # While true that there's more pages...
                    title      = browser.find_element_by_class_name('Topic-title').text
                    dates      = [e.get_attribute('data-tooltip-content') for e in browser.find_elements_by_class_name('TopicPost-timestamp') if e.text != ' (Edited)']
                    words      = [e.text for e in browser.find_elements_by_class_name('TopicPost-bodyContent')]
                    nums_dict  = [e.get_attribute('data-topic-post') for e in browser.find_elements_by_class_name('TopicPost')]
                    #sleep(2)
                    post_num   = [e.get_attribute('id') for e in browser.find_elements_by_class_name('TopicPost')]
                    auth_posts = [e.text for e in browser.find_elements_by_class_name('Author-posts') if e.get_attribute('data-toggle') == 'tooltip']
                    prof_link  = [e.get_attribute('href') for e in browser.find_elements_by_class_name('Author-avatar ')]
                    all_imgs   = [e.get_attribute('src') for e in browser.find_elements_by_css_selector('img')]
                    auth_img   = [e for e in all_imgs if 'blznav' not in e and len(e) != 0][1:-1]
                    #print('dict components defined')
                    for post in range(len(words)):                      # For each post in topic
                        try:
                            post_dict = {                               # Creation & Statement of dicts
                                'text'      : words[post],              # Text of the post
                                'date'      : dates[post],              # Date of the (unedited) post
                                'nums_dict' : nums_dict[post],          # Author info & votes of the post
                                'post_num'  : post_num[post],           # Post umber in the topic
                                'prof_link' : prof_link[post],          # Link to author profile
                                'auth_img'  : auth_img[post],           # Profile image of author
                                'title'     : title,                    # Title of Topic
                                'forum_page': forum_page}               # Page in the forum
                        except:
                            pass
                        try:
                            post_dict['auth_posts'] = auth_posts[post]  # Number of posts author has made
                        except:
                            pass
                        list_of_dicts.append(post_dict)
                    more_pages = click_next()                           # Clicks "NEXT" otherwise return False
                    #print('next clicked')
                browser.get(current_list)                               # Returns to current forum page
                #sleep(3)                                              # Tiny safety sleep
                #print('slept')
                title_list = browser.find_elements_by_class_name("ForumTopic-title") # Redefine list for freshness
                
            except:
                print("Woops", forum_page, i)                           # Informs of the problem
                save_posts(list_of_dicts, saver, forum_page)            # Save current posts
                sleep(10)                                               # Safety sleep
                browser.refresh()                                       # Safety Refresh
                browser.get(current_list)                               # Return to forum page
                title_list = browser.find_elements_by_class_name("ForumTopic-title") # Redefine for freshness
    except:
        print("Woops", forum_page)                                      # Informs that there's a problem
        save_posts(list_of_dicts, saver, forum_page)                    # Save current posts
        sleep(30)                                                       # Long Safety Sleep
        browser.refresh()                                               # Safety Refresh
        browser.get(current_list)                                       # Return to forum page
        
save_posts(list_of_dicts, saver, forum_page)                            # Finally save the posts
df = pd.DataFrame(list_of_dicts, columns = ['text','date'])             # Return the dataframe for looking! :D

Woops 0 20
Woops 0


TimeoutException: Message: timeout
  (Session info: chrome=71.0.3578.98)
  (Driver info: chromedriver=2.43.600229 (3fae4d0cda5334b4f533bede5a4787f7b832d052),platform=Mac OS X 10.14.0 x86_64)


In [315]:
def click_next():
    more_pages = True
    try:
        nexts = []
        for element in browser.find_elements_by_class_name('Button-content'):
            if element.text == 'NEXT':
                nexts.append(element)
        nexts[1].click()
        #not_last_page = True
    except:
        more_pages = False
    return more_pages

In [438]:
#ops = webdriver.chrome.options.Options()
#ops.add_argument('--dns-prefetch-disable')
path = '../Garage/chromedriver'                                         # Path to Chromedriver
browser = webdriver.Chrome(executable_path = path)#, options=ops)         # Open browser
list_of_dicts = []
saver = './data/full_scrapes/Overwatch_Test_' 

for forum_page in range(1,1000):
    try:
        current_list = 'https://us.battle.net/forums/en/overwatch/22813879/?page='+ str(forum_page)
        browser.get(current_list)
        title_list = browser.find_elements_by_class_name("ForumTopic-title")
        if forum_page % 100 == 0:
            save_posts(list_of_dicts, saver, forum_page)

        for i in range(len(title_list)):                                # For every title in the list
            try:                                                        # Failsafe in topic page
                title_list[i].click()                                   # Click on Title i
                more_pages = True                                       # True means there's more pages
                next_page = 2
                current_topic = browser.current_url
                while more_pages == True:                               # While true that there's more pages...
                    soup       = BeautifulSoup(browser.page_source, 'lxml')         # Soup of words
                    title      = soup.find(attrs={'class':'Topic-title'}).contents
                    TopicPosts = soup.find_all(attrs = {"class" :'TopicPost'})
                    dates      = [e for e in soup.find_all('a',{"class" :'TopicPost-timestamp'}) if e.contents != ['\n\t\t\t\t\t\t\t\xa0(Edited)\n']]
                    words      = soup.find_all('div',{"class" :'TopicPost-bodyContent'})
                    auth_posts = [e.contents for e in soup.find_all('a',attrs = {'class' :'Author-posts'})]
                    prof_link  = soup.find_all(attrs = {"class" :'Author-avatar '})
                    all_imgs   = [e.get_attribute('src') for e in browser.find_elements_by_css_selector('img')]
                    auth_img   = [e for e in all_imgs if 'blznav' not in e and len(e) != 0][1:-1]
                    for post in range(len(words)):                      # For each post in topic
                        try:
                            post_dict = {                               # Creation & Statement of dicts
                                'text'      : words[post].contents,                     # Text of the post
                                'date'      : dates[post].attrs['data-tooltip-content'],# Date of the (unedited) post
                                'ids_dict'  : TopicPosts[post].attrs['data-topic-post'],# Author info & votes of the post
                                'post_num'  : TopicPosts[post].attrs['id'] ,            # Post number in the topic
                                'statuses'  : TopicPosts[post].attrs['data-topic'],
                                'prof_link' : prof_link[post].attrs['href'],            # Link to author profile
                                'auth_img'  : auth_img[post],           # Profile image of author
                                'title'     : title,                    # Title of Topic
                                'forum_page': forum_page}               # Page in the forum
                        except:
                            pass
                        try:
                            post_dict['auth_posts'] = auth_posts[post]  # Number of posts author has made
                        except:
                            pass
                        list_of_dicts.append(post_dict)
                    try:
                        browser.get(current_topic + '?page=' + str(next_page))
                        soup = BeautifulSoup(browser.page_source, 'lxml')
                        if soup.find(attrs = {'class':'error-type'}).contents[0] == '404':
                            more_pages = False
                        else:
                            next_page += 1
                    except:
                        more_pages = False
                browser.get(current_list)                               # Returns to current forum page
                #sleep(3)                                                # Tiny safety sleep
                title_list = browser.find_elements_by_class_name("ForumTopic-title") # Redefine list for freshness
                
            except:
                print("Woops", forum_page, i)                           # Informs of the problem
                save_posts(list_of_dicts, saver, forum_page)            # Save current posts
                sleep(10)                                               # Safety sleep
                browser.refresh()                                       # Safety Refresh
                browser.get(current_list)                               # Return to forum page
                title_list = browser.find_elements_by_class_name("ForumTopic-title") # Redefine for freshness
    except:
        print("Woops", forum_page)
        save_posts(list_of_dicts, saver, forum_page)
        sleep(30)
        browser.refresh()
        browser.get(current_list)
save_posts(list_of_dicts, saver, forum_page)
df = pd.DataFrame(list_of_dicts, columns = ['text','date'])        

Woops 1


KeyboardInterrupt: 

In [450]:
# THIS is our new official one??
path = '../Garage/chromedriver'                                         # Path to Chromedriver
browser = webdriver.Chrome(executable_path = path)#, options=ops)         # Open browser
list_of_dicts = []
saver = './data/full_scrapes/Overwatch_Test_' 

for forum_page in range(72,1000):
    try:
        current_list = 'https://us.battle.net/forums/en/overwatch/22813879/?page='+ str(forum_page)
        browser.get(current_list)
        title_list = browser.find_elements_by_class_name("ForumTopic-title")
        if forum_page % 100 == 0:
            save_posts(list_of_dicts, saver, forum_page)

        for i in range(len(title_list)):                                # For every title in the list
            try:                                                        # Failsafe in topic page
                title_list[i].click()                                   # Click on Title i
                more_pages = True                                       # True means there's more pages
                next_page = 2
                current_topic = browser.current_url
                while more_pages == True:                               # While true that there's more pages...
                    soup       = BeautifulSoup(browser.page_source, 'lxml')         # Soup of words
                    title      = soup.find(attrs={'class':'Topic-title'}).contents
                    TopicPosts = soup.find_all(attrs = {"class" :'TopicPost'})
                    dates      = [e for e in soup.find_all('a',{"class" :'TopicPost-timestamp'}) if e.contents != ['\n\t\t\t\t\t\t\t\xa0(Edited)\n']]
                    words      = soup.find_all('div',{"class" :'TopicPost-bodyContent'})
                    auth_posts = [e.contents for e in soup.find_all('a',attrs = {'class' :'Author-posts'})]
                    prof_link  = soup.find_all(attrs = {"class" :'Author-avatar '})
                    all_imgs   = [e.get_attribute('src') for e in browser.find_elements_by_css_selector('img')]
                    auth_img   = [e for e in all_imgs if 'blznav' not in e and len(e) != 0][1:-1]
                    for post in range(len(words)):                      # For each post in topic
                        try:
                            post_dict = {                               # Creation & Statement of dicts
                            'text'      : words[post].contents,                     # Text of the post
                            'date'      : dates[post].attrs['data-tooltip-content'],# Date of the (unedited) post
                            'ids_dict'  : TopicPosts[post].attrs['data-topic-post'],# Author info & votes of the post
                            'post_num'  : TopicPosts[post].attrs['id'] ,            # Post number in the topic
                            'auth_img'  : auth_img[post],           # Profile image of author
                            'title'     : title,                    # Title of Topic
                            'forum_page': forum_page}               # Page in the forum
                        except:
                            pass
                        try:
                            post_dict['statuses']   = TopicPosts[post].attrs['data-topic']
                            post_dict['prof_link']  = prof_link[post].attrs['href']
                            post_dict['auth_posts'] = auth_posts[post]  # Number of posts author has made
                        except:
                            pass
                        list_of_dicts.append(post_dict)
                    try:
                        browser.get(current_topic + '?page=' + str(next_page))
                        soup = BeautifulSoup(browser.page_source, 'lxml')
                        #print('attempted new page')
                        if soup.find(attrs = {'class':'error-type'}).contents[0] == '404':
                            #print("got a 404")
                            more_pages = False
                        #else:
                        #    print("there is another page and we added 1")
                        #    next_page += 1
                    except:
                        next_page += 1
                browser.get(current_list)                               # Returns to current forum page
                #sleep(3)                                                # Tiny safety sleep
                title_list = browser.find_elements_by_class_name("ForumTopic-title") # Redefine list for freshness
            except KeyboardInterrupt:
                print("Canceled!")
            except:
                print("TimedOut on Forum Section:", forum_page,"Topic number:", i, "Topic Page:", next_page-1)                           # Informs of the problem
                save_posts(list_of_dicts, saver, forum_page)            # Save current posts
                sleep(10)                                               # Safety sleep
                browser.refresh()                                       # Safety Refresh
                browser.get(current_list)                               # Return to forum page
                title_list = browser.find_elements_by_class_name("ForumTopic-title") # Redefine for freshness
            #except:
            #    print('SOMETHING ELSE WENT WRONG')
            #    pass
    except KeyboardInterrupt:
        print("Canceled!")
    except:
        print("TimedOut on Forum Section:", forum_page)
        save_posts(list_of_dicts, saver, forum_page)
        sleep(30)
        browser.refresh()
        browser.get(current_list)
    #except:
    #    print("SOMETHING ELSE WENT WRONG")
    #    pass
save_posts(list_of_dicts, saver, forum_page)
df = pd.DataFrame(list_of_dicts, columns = ['text','date'])        

TimedOut on Forum Section: 72 Topic number: 32 Topic Page: 1
TimedOut on Forum Section: 72


TimeoutException: Message: timeout
  (Session info: chrome=71.0.3578.98)
  (Driver info: chromedriver=2.43.600229 (3fae4d0cda5334b4f533bede5a4787f7b832d052),platform=Mac OS X 10.14.0 x86_64)


In [422]:
[e.contents for e in soup.find_all('a',{"class" :'TopicPost-timestamp'}) if e.contents != ['\n\t\t\t\t\t\t\t\xa0(Edited)\n']]

[['\n\t\t\t\t\t\tMay 24, 2016\n\t\t\t\t\t'],
 ['\n\t\t\t\t\t\tMay 24, 2016\n\t\t\t\t\t'],
 ['\n\t\t\t\t\t\tMay 24, 2016\n\t\t\t\t\t'],
 ['\n\t\t\t\t\t\tMay 24, 2016\n\t\t\t\t\t'],
 ['\n\t\t\t\t\t\tMay 24, 2016\n\t\t\t\t\t'],
 ['\n\t\t\t\t\t\tMay 24, 2016\n\t\t\t\t\t'],
 ['\n\t\t\t\t\t\tMay 24, 2016\n\t\t\t\t\t'],
 ['\n\t\t\t\t\t\tMay 24, 2016\n\t\t\t\t\t'],
 ['\n\t\t\t\t\t\tMay 24, 2016\n\t\t\t\t\t'],
 ['\n\t\t\t\t\t\tMay 24, 2016\n\t\t\t\t\t'],
 ['\n\t\t\t\t\t\tMay 24, 2016\n\t\t\t\t\t'],
 ['\n\t\t\t\t\t\tMay 24, 2016\n\t\t\t\t\t'],
 ['\n\t\t\t\t\t\tMay 24, 2016\n\t\t\t\t\t'],
 ['\n\t\t\t\t\t\tMay 24, 2016\n\t\t\t\t\t'],
 ['\n\t\t\t\t\t\tMay 24, 2016\n\t\t\t\t\t'],
 ['\n\t\t\t\t\t\tMay 24, 2016\n\t\t\t\t\t'],
 ['\n\t\t\t\t\t\tMay 24, 2016\n\t\t\t\t\t'],
 ['\n\t\t\t\t\t\tMay 24, 2016\n\t\t\t\t\t'],
 ['\n\t\t\t\t\t\tMay 24, 2016\n\t\t\t\t\t'],
 ['\n\t\t\t\t\t\tMay 24, 2016\n\t\t\t\t\t']]

In [4]:
# THIS is just before deleting a bunch of junk
list_of_dicts = []
saver = './data/full_scrapes/Overwatch_Test_' 

for forum_page in range(0,100):
    #try:
    topic_keys = urls_with_numbers('https://us.battle.net/forums/en/overwatch/22813879/?page='+ str(forum_page))
    if forum_page % 100 == 0:
        save_posts(list_of_dicts, saver, forum_page) 
    for topic_stats in topic_keys:                                # For every title in the list
        for page in range(topic_stats[1]):
            page += 1               # TRY "for page +1 in range(topic_stats[1]):" later
            if page > 1:
                URL = topic_stats[0] + '?page=' + str(page)
            else:
                URL = topic_stats[0]
            page_soup  = get_html(URL)         # Soup of words
            title      = page_soup.find(attrs={'class':'Topic-title'}).contents
            dates      = [e for e in page_soup.find_all('a',{"class" :'TopicPost-timestamp'}) if e.contents != ['\n\t\t\t\t\t\t\t\xa0(Edited)\n']]
            words      = page_soup.find_all('div',{"class" :'TopicPost-bodyContent'})
            TopicPosts = page_soup.find_all(attrs = {"class" :'TopicPost'})
            auth_posts = [e.contents for e in page_soup.find_all('a',attrs = {'class' :'Author-posts'})]
            prof_link  = page_soup.find_all(attrs = {"class" :'Author-avatar '})
        for page in range(topic_info[1]):                      # For each post in topic

            turn_to_dict(post)
            list_of_dicts.append(post_dict)
        try:
            resoup = get_html(title_list[i] + '?page=' + str(next_page))
            if resoup.find(attrs = {'class':'error-type'}).contents[0] == '404':
                more_pages = False
        except:
            next_page += 1
        resoup = get_html(current_list)                               # Returns to current forum page
        #except KeyboardInterrupt:
        #    print("Canceled!")
        #except:
        #    print("TimedOut on Forum Section:", forum_page,"Topic number:", i, "Topic Page:", next_page-1)                           # Informs of the problem
        #    save_posts(list_of_dicts, saver, forum_page)            # Save current posts
        #    sleep(10)    
        print("Success!:", forum_page,"Topic number:", i, "Topic Page:", next_page-1)
    #except KeyboardInterrupt:
    #    print("Canceled!")
    #except:
    #    print("TimedOut on Forum Section:", forum_page)
    #    save_posts(list_of_dicts, saver, forum_page)
    #    sleep(30)
    #    resoup = get_html(current_list)
save_posts(list_of_dicts, saver, forum_page)
df = pd.DataFrame(list_of_dicts, columns = ['text','date'])        

AttributeError: 'NoneType' object has no attribute 'contents'

In [413]:
for i in soup.find_all('a',{"class" :'TopicPost-timestamp'}):
    print(i)

<a class="TopicPost-timestamp" data-toggle="tooltip" data-tooltip-content="05/24/2016 12:44 AM" href="#post-1">
						May 24, 2016
					</a>
<a class="TopicPost-timestamp" data-toggle="tooltip" data-tooltip-content="05/24/2016 12:45 AM" href="#post-2">
						May 24, 2016
					</a>
<a class="TopicPost-timestamp" data-toggle="tooltip" data-tooltip-content="MartinRiggs 05/24/2016 12:46 AM" href="#post-2">
							 (Edited)
</a>
<a class="TopicPost-timestamp" data-toggle="tooltip" data-tooltip-content="05/24/2016 12:46 AM" href="#post-3">
						May 24, 2016
					</a>
<a class="TopicPost-timestamp" data-toggle="tooltip" data-tooltip-content="05/24/2016 12:46 AM" href="#post-4">
						May 24, 2016
					</a>
<a class="TopicPost-timestamp" data-toggle="tooltip" data-tooltip-content="05/24/2016 12:47 AM" href="#post-5">
						May 24, 2016
					</a>
<a class="TopicPost-timestamp" data-toggle="tooltip" data-tooltip-content="05/24/2016 12:48 AM" href="#post-6">
						May 24, 2016
					</a>
<a class

http://cryptocats.thetwentysix.io/
    1.) Scrape entire page of cats including
        Cat #
        Cat Status (claimed or not)
        Cat URL
    2.)Identify the navigation items "origin 12", Austrailian Blockchain" and "December"
    3.) Select each of these buttons, automate a click action to each of them
    4.) Run your scraping process from #1, appending the "release type", corresponding to each of these release filters

In [ ]:
# Scrape this for a potential starting list of curse words for a list of cuss words
https://www.noswearing.com/dictionary/a
Tweet Sentiment Dataset
https://www.kaggle.com/kazanova/sentiment140/home